In [ ]:
from preprocess.preprocess import load_dataset, compute_label_agg, select_data, get_data_loaders

import torch
import torch.nn as nn
import torch.optim as optim

from models.RNN import RNN
from utils.RNN_utils import *

In [ ]:
# Define device for torch
use_cuda = True
print("CUDA is available:", torch.cuda.is_available())
device = torch.device("cuda" if (
    use_cuda and torch.cuda.is_available()) else "cpu")

# Define options

In [ ]:
LOAD_DATASET = False
TRAIN_PATH = './PTB_train.pt'
VALID_PATH = './PTB_valid.pt'
TEST_PATH = './PTB_test.pt'


# Loading the dataset

In [ ]:
if LOAD_DATASET:
    train_loader = torch.load(TRAIN_PATH)
    valid_loader = torch.load(VALID_PATH)
    test_loader = torch.load(TEST_PATH)
else:
    path = './data/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3/'

    data, raw_labels = load_dataset(path)

    labels = compute_label_agg(raw_labels, path)

    data, labels, y = select_data(data, labels)

    train_loader, valid_loader, test_loader = get_data_loaders(
        data, labels, y, 256)

    torch.save(train_loader, TRAIN_PATH)
    torch.save(valid_loader, VALID_PATH)
    torch.save(test_loader, TEST_PATH)

# Training the model

In [ ]:
EPOCH = 30

model = RNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.BCEWithLogitsLoss()

model = train_RNN(EPOCH, train_loader, valid_loader, model,
                  criterion, optimizer, device=device)

# Evaluate and plot losses

In [ ]:
evaluate_RNN(model, test_loader)
plot_losses()
